<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/Cocacola_%26_Pepsico_2025_Daily_Candlestick_Charts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# KO & PEP 2025 Daily Candlestick Charts -> PNG
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

def load_ohlcv(csv_file):
    """Load CSV in Stooq or Yahoo format and normalize columns."""
    if not os.path.exists(csv_file):
        raise FileNotFoundError(f"File not found: {csv_file}")

    df = pd.read_csv(csv_file)
    df.columns = [c.strip().lower() for c in df.columns]

    if 'date' not in df.columns:
        raise ValueError(f"{csv_file}: CSV must contain a 'Date' column.")

    # Ensure essential columns exist
    needed = ['open', 'high', 'low', 'close', 'volume']
    for col in needed:
        if col not in df.columns:
            raise ValueError(f"{csv_file}: missing required column '{col}'.")

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    return df[['date', 'open', 'high', 'low', 'close', 'volume']]

def filter_year(df, year=2025):
    start = f"{year}-01-01"
    end   = f"{year}-12-31"
    m = (df['date'] >= start) & (df['date'] <= end)
    out = df.loc[m].copy()
    if out.empty:
        raise ValueError(f"No rows found for {year}.")
    return out

def plot_candlestick_2025(df_yr, title, outfile):
    # Compute MAs on the filtered year slice (display-only)
    df_yr['ma50'] = df_yr['close'].rolling(50).mean()
    df_yr['ma200'] = df_yr['close'].rolling(200).mean()

    dates = mdates.date2num(np.array(df_yr['date'].dt.to_pydatetime()))
    ohlc = list(zip(dates, df_yr['open'], df_yr['high'], df_yr['low'], df_yr['close']))

    plt.style.use('seaborn-v0_8-whitegrid')
    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_axes([0.06, 0.25, 0.93, 0.70])
    ax_vol = fig.add_axes([0.06, 0.08, 0.93, 0.13], sharex=ax)

    # Candlesticks
    for d, o, h, l, c in ohlc:
        color = '#2ca02c' if c >= o else '#d62728'
        ax.vlines(d, l, h, color=color, linewidth=1)
        body_width = 0.7
        ax.add_patch(plt.Rectangle((d - body_width/2, min(o, c)),
                                   body_width, max(1e-8, abs(c - o)),
                                   facecolor=color, edgecolor=color, linewidth=0.5))

    # MAs
    ax.plot(df_yr['date'], df_yr['ma50'], color='#1f77b4', linewidth=1.8, label='MA 50')
    ax.plot(df_yr['date'], df_yr['ma200'], color='#ff7f0e', linewidth=1.8, label='MA 200')

    # High/Low
    yr_high = df_yr['high'].max()
    yr_low = df_yr['low'].min()
    d_high = df_yr.loc[df_yr['high'].idxmax(), 'date']
    d_low = df_yr.loc[df_yr['low'].idxmin(), 'date']
    ax.scatter([d_high, d_low], [yr_high, yr_low], color='black', zorder=3, s=50)
    ax.annotate(f'2025 High: {yr_high:,.2f}\n({d_high.date()})', xy=(d_high, yr_high),
                xytext=(30, 25), textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', fc='lightgreen', alpha=0.7))
    ax.annotate(f'2025 Low: {yr_low:,.2f}\n({d_low.date()})', xy=(d_low, yr_low),
                xytext=(-60, -40), textcoords='offset points', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', fc='salmon', alpha=0.7))

    # Axes formatting
    ax.set_title(title, fontsize=16)
    ax.set_ylabel('Price (USD)', fontsize=12)
    ax.legend(loc='upper left', frameon=True, fontsize=10)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
    ax.grid(True, linestyle='--', alpha=0.5)

    # Volume
    ax_vol.bar(df_yr['date'], df_yr['volume'], width=0.8, color='#7f7f7f')
    ax_vol.set_ylabel('Volume', fontsize=12)
    ax_vol.grid(True, linestyle='--', alpha=0.5)

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(outfile, dpi=250)
    plt.close(fig)
    print(f"Saved: {outfile}")

# Run for KO and PEP
datasets = [
    ("ko_us_d.csv", "KO — Daily Candlesticks (Jan 1, 2025 → Dec 31, 2025)", "ko_2025_candlesticks.png"),
    ("pep_us_d.csv", "PEP — Daily Candlesticks (Jan 1, 2025 → Dec 31, 2025)", "pep_2025_candlesticks.png"),
]

for csv_file, title, outfile in datasets:
    df_all = load_ohlcv(csv_file)
    df_2025 = filter_year(df_all, 2025)
    plot_candlestick_2025(df_2025, title, outfile)



/tmp/ipython-input-3564162537.py:43: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  dates = mdates.date2num(np.array(df_yr['date'].dt.to_pydatetime()))
/tmp/ipython-input-3564162537.py:91: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])


Saved: ko_2025_candlesticks.png


/tmp/ipython-input-3564162537.py:43: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  dates = mdates.date2num(np.array(df_yr['date'].dt.to_pydatetime()))
/tmp/ipython-input-3564162537.py:91: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])


Saved: pep_2025_candlesticks.png
